In [0]:
def aldsconnector():
    spark.conf.set("fs.azure.account.key.optumsadls.dfs.core.windows.net",dbutils.secrets.get(scope="optumscope", key="optumadlskey"))
    print("adls connection successful")

In [0]:
def blobconnector():
    spark.conf.set("fs.azure.account.key.optumblob.dfs.core.windows.net",dbutils.secrets.get(scope="optumscope", key="optumblobkey"))
    print("blob connection successful")

In [0]:
def readbronzedata(fn):
    aldsconnector()
    df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://optum@optumsadls.dfs.core.windows.net/bronze/"+fn)
    return df

In [0]:
def readbronzedatajson(fn):
    aldsconnector()
    df = spark.read.format("json").load("abfss://optum@optumsadls.dfs.core.windows.net/bronze/"+fn)
    return df

In [0]:
def readsilverdata(fn):
    aldsconnector()
    df = spark.read.format("csv").option("header", "true").load("abfss://optum@optumsadls.dfs.core.windows.net/silver/"+fn)
    return df

In [0]:
def readgolddata(fn):
    aldsconnector()
    df = spark.read.format("csv").option("header", "true").load("abfss://optum@optumsadls.dfs.core.windows.net/gold/"+fn)
    return df

In [0]:
def write2silver(df, file_name):
    silver_path = "abfss://optum@optumsadls.dfs.core.windows.net/silver/"
    temp_path = f"{silver_path}/output_temp"
    final_path = f"{silver_path}/{file_name}"
    df.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path)
    files = dbutils.fs.ls(temp_path)
    csv_file = [f.path for f in files if f.path.endswith(".csv")][0]
    dbutils.fs.mv(csv_file, final_path)
    dbutils.fs.rm(temp_path, recurse=True)
    print(f"CSV file successfully written to {final_path}")
    return final_path

In [0]:
def write2gold(df, file_name):
    gold_path = "abfss://optum@optumsadls.dfs.core.windows.net/gold/"
    temp_path = f"{gold_path}/output_temp"
    final_path = f"{gold_path}/{file_name}"
    df.coalesce(1).write.mode("overwrite").option("header", "true").csv(temp_path)
    files = dbutils.fs.ls(temp_path)
    csv_file = [f.path for f in files if f.path.endswith(".csv")][0]
    dbutils.fs.mv(csv_file, final_path)
    dbutils.fs.rm(temp_path, recurse=True)
    print(f"CSV file successfully written to {final_path}")
    return final_path